In [0]:
import sys, os
sys.path.append(os.path.abspath('/Workspace/Repos/zhastay_yeltay@epam.com/utils/'))

from delta.tables import *
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DateType, LongType

from init import source_path, working_path, bronze, silver, gold
from init import init_spark
from udfs import * 
init_spark()

from util_logger import init_logger
dbutils.widgets.text('task', "test_logger")
logger = init_logger(dbutils.widgets.get('task'))

In [0]:
!pip install folium

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import folium
import random


data = (
    spark
    .table("hive_metastore.zhastay_yeltay_03_gold.01v2_cities_by_vip_customer_count")
    .select("lat", "lng", F.col('total_vip_customers').alias("cnt"), "city")
    .orderBy(F.col('cnt').desc())
    .filter(
        (F.col('city') != 'Hamilton')
        & (F.col('cnt') > 250)
    )
    # .limit(50)
    .collect()
)

def random_color():
    return '#{:06x}'.format(random.randint(0, 0xFFFFFF))

map_center_lat = sum(lat for lat, lon, cnt, city in data) / len(data)
map_center_lon = sum(lon for lat, lon, cnt, city in data) / len(data)
mymap = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=7)


for row in data:
    popup_text = f"City: {row['city']}<br> VIP: {row['cnt']}"
    folium.Circle(
        location=[row['lat'], row['lng']],
        radius=(row['cnt'] * 10), 
        color=random_color(),
        fill=True,
        fill_color=random_color(),
        popup=popup_text
    ).add_to(mymap)

display(mymap)

In [0]:
%sql

SELECT
*
FROM hive_metastore.zhastay_yeltay_03_gold.01v2_cities_by_vip_customer_count

lat,lng,city,total_vip_customers
47.3934177987,-122.2319639731,Kent,1505
47.605715454,-122.176202523,Bellevue,553
47.2258538219,-121.4073351011,Auburn,1082
45.5213691577,-122.641119307,Portland,2759
45.5102852438,-122.9328596038,Hillsboro,999
48.0995014183,-122.8094707511,Port Townsend,1723
43.5421257625,-105.0989799175,Hamilton,250
44.5692511477,-123.2748243469,Corvallis,803
43.796751294,-123.049602703,Cottage Grove,305
30.9922595,-86.33686305,Laurel Hill,126


In [0]:
%sql

SELECT
*
FROM zhastay_yeltay_03_gold.01_cities_by_vip_customer_count
ORDER BY
    total_vip_customers DESC
LIMIT 5

city,total_vip_customers
Seattle,4637
Portland,2759
Kirkland,2744
Port Townsend,1723
Kent,1505


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

SELECT
  state,
  is_metropolitan,
  sum(coalesce(cnt, 0)) AS cnt,
  sum(coalesce(total_revenue, 0)) AS total_revenue
FROM zhastay_yeltay_03_gold.total_revenue_by_all_categories
GROUP BY
  state,
  is_metropolitan
  

state,is_metropolitan,cnt,total_revenue
Hawaii,true,59923,229351
Hawaii,false,25686,98194
Alabama,true,357935,1369564
Oregon,false,1689638,6462932
Washington,true,12751183,48747604
Oregon,true,5017956,19164579
Alabama,false,186561,712288
Washington,false,12542591,47960201


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

SELECT
*
FROM zhastay_yeltay_03_gold.`05_affiliate_by_weekly_orders`

customer_id,customer_status
58210,Regular
67808,Regular
31662,Regular
48557,Regular
93199,Regular
8136,Regular
98801,Regular
73608,Regular
56077,Regular
50605,Regular


In [0]:
%sql

SELECT
delivered_on,
revenue AS val
FROM zhastay_yeltay_03_gold.daily_total_revenue

delivered_on,val
2010-04-05,123.18
2010-04-15,36884.60
2010-04-20,110370.26
2010-04-21,3680.38
2010-04-26,165142.83
2010-04-30,33043.95
2010-05-05,288001.03
2010-05-06,91881.26
2010-05-10,179660.47
2010-05-11,339092.48


Databricks visualization. Run in Databricks to view.

In [0]:
%sql

SELECT
  state,
  city,
  COUNT(*) FILTER(WHERE not reached_at_least_5_orders_all_weeks)::NUMERIC AS total_inactive_customers,
  COUNT(*) AS total_customers
FROM zhastay_yeltay_03_gold.customers_havent_reached_at_least_5_orders_all_weeks
GROUP BY
  state,
  city


state,city,total_inactive_customers,total_customers
Oregon,Clackamas,0,2564
Oregon,Salem,0,3291
Washington,Navy Indian Island,0,795
Oregon,Sunriver,0,2240
Alabama,Madison,0,741
Washington,Kent,82,20924
Alabama,Dothan,0,1785
Oregon,Sandy,0,1476
Washington,Issaquah,5,4448
Alabama,Daulphin Island,0,2444


Databricks visualization. Run in Databricks to view.